In [50]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)

In [51]:
print('Loading Data')
matched_volumes = pd.read_csv('../input/matched_hathi_estc_nov2023.csv', index_col=0)
estc_data = pd.read_csv('../input/estc_1500_to_1800.csv', index_col=0)
hathitrust_data = pd.read_csv('../temporary/volumes_scores.csv')
metadata = pd.read_csv('../temporary/metadata.csv')

Loading Data


C:\Users\Austin\AppData\Local\Temp\ipykernel_20088\2579523277.py:3: DtypeWarning: Columns (29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  estc_data = pd.read_csv('../input/estc_1500_to_1800.csv', index_col=0)


In [52]:
matched_volumes.head()

,Title,Title.1,HTID,ESTC System No.,Match Score
0,A sentimental journey through France and Italy,Yorickâ€™s sentimental journey through France ...,coo.31924096960947,6214773,87.378641
1,Traditions of Eden,The Garden of Eden,coo.31924096961259,6157289,61.111111
2,A history of the county of Brecknock,A list of the court of aldermen,coo.31924096162064,6159287,71.641791
3,"Tree and serpent worship: or, Illustrations of...",The history of the Church of Christ. Volume th...,coo.31924096988872,6059892,56.170213
4,Thirteen satires,The snare,coo.31924096977644,6133567,64.000000


In [53]:
estc_data.rename(columns={'Title': 'title_estc', 'ESTC System No.': 'estc_id'}, inplace=True)
estc_data.head()

,estc_id,ESTC Citation No.,Author - personal,title_estc,Publisher/year,Physical descr.,General note,Citation/references,Surrogates,Loc. of filmed copy,Subject,Added name,Copies - Brit.Isles,Copies - N.America,Electronic location,Variant title,Uncontrolled note,Author - corporate,Uniform title,Corporate subject,Copies - Other,Edition,Person as subject,Added Title,Genre/form,Title as subject,Copies - Europe,Collective title,Conference subject,Index/finding aids,Numbering note,Dates of pub.,Publicat. frequency,Continued by,Other editions,Supplement to,Link note,Related item,Supplement details,Continues,Meeting name,Supplement note,System details,Issued with,Translated as,Loc. of original,"Summary, etc.",Former pub. freq.,Abbreviated title,Translation of,Issuing body,Merged with,Absorbed in part,Bibliography etc.,Series,Absorbed by,Continued in pt. by,Related record,Absorbed,Formed by,Changed back to,Continues in part,Geographic coverage,Separated from,Language note
0,6182456,S110939,"Betson, Thomas.",Here begynneth a ryght profytable treatyse co[...,"[Westminster : Wynkyn de Worde, 1500]",[36] p. : ill. ; 4⁰.,"['Caption title.', 'Imprint from STC.', 'Print...","['STC (2nd ed.), 1978', 'Duff, 43', 'GW, 4190']","Available on microfilm. Ann Arbor, Mich.: Univ...",C.,Christian life -- Catholic authors -- Early wo...,"Worde, Wynkyn de, -1534?, printer.",['Cambridge University Library (includes Sir G...,"['Folger Shakespeare', 'United States, Library...",['Early English Books Online (EEBO) ; { Reprod...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6190320,S118960,NaN,[The history of Guy of Warwick],"[London? : [R. Pynson, 1500?]]",[3]+ leaves ; 4⁰.,['Printer’s name from and publication date con...,"STC (2nd ed.), 12540","Available on microfilm. Ann Arbor, Mich.: Univ...",L.,Guy of Warwick (Legendary character) -- Early ...,"Pynson, Richard, -1530, printer.","['British Library', 'British Library']",NaN,Early English Books Online (EEBO) ; { Reproduc...,"['Erle to be his leue home agayne', 'Erle to b...",RPM 10/18/96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6203003,S93554,NaN,[Horae ad usum Sarum.],[Thys emprynteth at Westmynster : by me Julyay...,[32]+ p. ; 64⁰.,"['Title from STC.', 'Imprint from colophon.', ...","STC (2nd ed.), 15895",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Catholic Church.,Liturgies. Latin. Hours and Primers. Salisbury...,Catholic Church -- Liturgies -- Early works to...,State Library of Victoria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6186462,S115128,NaN,[Sir Eglamour],"[Westminster : Wynkyn de Worde, 1500]",[2+] p. ; 4⁰.,['In verse. Second line on only extant leaf re...,"['STC (2nd ed.), 7541', 'Duff, 135']","Available on microfilm. Ann Arbor, Mich.: Univ...",C.,Romances -- Early works to 1800.,"Worde, Wynkyn de, -1534?, printer.",Cambridge University Library (includes Sir Geo...,NaN,Early English Books Online (EEBO) ; { Reproduc...,If that he be a crysten man,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6204328,S92225,NaN,[11 Edw. IV.] De termino sancte Trinitatis [etc.],"London [R. Pynson, 1500?]",[No pagination provided] ; 2⁰.,a-b⁶ c⁴ (a1 blank); not foliated; 16 leaves.,"STC (2nd ed.), 9836",NaN,NaN,NaN,"Pynson, Richard, -1530, printer.","The John Rylands Library, The University of Ma...",NaN,NaN,NaN,Place-holder record.,NaN,ENGLAND - YEARBOOKS - Edward IV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [54]:
#clean estc metadata
characters_to_remove = "[]()-'"

# #make new column
# estc_data[['Subject', 'Corporate subject', 'Person as subject', 'Title as subject', 'Conference subject']] = estc_data[['Subject', 'Corporate subject', 'Person as subject', 'Title as subject', 'Conference subject']].astype(str)

conditions = [
    ~estc_data['Subject'].isna(),
    estc_data['Subject'].isna() & ~estc_data['Corporate subject'].isna()
]

choices = [
    estc_data['Subject'],
    estc_data['Corporate subject']
]

estc_data['Topic'] = np.select(conditions, choices, default='nan')


# estc_data['Subject'] = estc_data['Subject'].str.translate({ord(c): None for c in characters_to_remove})
estc_data


,estc_id,ESTC Citation No.,Author - personal,title_estc,Publisher/year,Physical descr.,General note,Citation/references,Surrogates,Loc. of filmed copy,Subject,Added name,Copies - Brit.Isles,Copies - N.America,Electronic location,Variant title,Uncontrolled note,Author - corporate,Uniform title,Corporate subject,Copies - Other,Edition,Person as subject,Added Title,Genre/form,Title as subject,Copies - Europe,Collective title,Conference subject,Index/finding aids,Numbering note,Dates of pub.,Publicat. frequency,Continued by,Other editions,Supplement to,Link note,Related item,Supplement details,Continues,Meeting name,Supplement note,System details,Issued with,Translated as,Loc. of original,"Summary, etc.",Former pub. freq.,Abbreviated title,Translation of,Issuing body,Merged with,Absorbed in part,Bibliography etc.,Series,Absorbed by,Continued in pt. by,Related record,Absorbed,Formed by,Changed back to,Continues in part,Geographic coverage,Separated from,Language note,Topic
0,6182456,S110939,"Betson, Thomas.",Here begynneth a ryght profytable treatyse co[...,"[Westminster : Wynkyn de Worde, 1500]",[36] p. : ill. ; 4⁰.,"['Caption title.', 'Imprint from STC.', 'Print...","['STC (2nd ed.), 1978', 'Duff, 43', 'GW, 4190']","Available on microfilm. Ann Arbor, Mich.: Univ...",C.,Christian life -- Catholic authors -- Early wo...,"Worde, Wynkyn de, -1534?, printer.",['Cambridge University Library (includes Sir G...,"['Folger Shakespeare', 'United States, Library...",['Early English Books Online (EEBO) ; { Reprod...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Christian life -- Catholic authors -- Early wo...
1,6190320,S118960,NaN,[The history of Guy of Warwick],"[London? : [R. Pynson, 1500?]]",[3]+ leaves ; 4⁰.,['Printer’s name from and publication date con...,"STC (2nd ed.), 12540","Available on microfilm. Ann Arbor, Mich.: Univ...",L.,Guy of Warwick (Legendary character) -- Early ...,"Pynson, Richard, -1530, printer.","['British Library', 'British Library']",NaN,Early English Books Online (EEBO) ; { Reproduc...,"['Erle to be his leue home agayne', 'Erle to b...",RPM 10/18/96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Guy of Warwick (Legendary character) -- Early ...
2,6203003,S93554,NaN,[Horae ad usum Sarum.],[Thys emprynteth at Westmynster : by me Julyay...,[32]+ p. ; 64⁰.,"['Title from STC.', 'Imprint from colophon.', ...","STC (2nd ed.), 15895",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Catholic Church.,Liturgies. Latin. Hours and Primers. Salisbury...,Catholic Church -- Liturgies -- Early works to...,State Library of Victoria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Catholic Church -- Liturgies -- Early works to...
3,6186462,S115128,NaN,[Sir Eglamour],"[Westminster : Wynkyn de Worde, 1500]",[2+] p. ; 4⁰.,['In verse. Second line on only extant leaf re...,"['STC (2nd ed.), 7541', 'Duff, 135']","Available on microfilm. Ann Arbor, Mich.: Univ...",C.,Romances -- Early works to 1800.,"Worde, Wynkyn de, -1534?, printer.",Cambridge University Library (includes Sir Geo...,NaN,Early English Books Online (EEBO) ; { Reproduc...,If that he be a crysten man,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Romances -- Early works to 1800.
4,6204328,S92225,NaN,[11 Edw. IV.] De termino sancte Trinitatis [etc.],"London [R. Pynson, 1500?]",[No pagination provided] ; 2⁰.,a-b⁶ c⁴ (a1 blank); not foliated; 16 leaves.,"STC (2nd ed.), 9836",NaN,NaN,NaN,"Pynson, Richard, -1530, printer.","The

In [6]:
hathitrust_data.head()

,Religion,Science,Political Economy,HTID,industry_2,industry_3,industry_1643,percent_regression,percent_pessimism,percent_optimistic,percent_progress_original,optimism_score,Year,oclc,Year_rounded,percent_progress_main,percent_progress_secondary,progress_regression_original,progress_regression_main,progress_regression_secondary,optimism_percentile,industry_2_percentile,industry_3_percentile,industry_1643_percentile,optimistic_percentile,progress_percentile_original,progress_percentile_main,progress_percentile_secondary,pessimism_percentile,regression_percentile,progress_regression_percentile_original,progress_regression_percentile_main,progress_regression_percentile_secondary
0,0.326938,0.164118,0.508944,uc1.b5568131,0.099625,0.101499,0.008745,0.0,0.000000,0.012804,0.000937,0.013741,1898.0,1883666.0,1898.0,0.000937,0.000937,0.000937,0.000937,0.000937,0.853506,0.342681,0.306506,0.200520,0.806700,0.565035,0.634121,0.695959,0.132470,0.490374,0.565902,0.635049,0.696899
1,0.410077,0.104824,0.485099,uc1.$b135547,0.151136,0.183048,0.022211,0.0,0.001021,0.016850,0.005617,0.021445,1832.0,8582654.0,1832.0,0.005617,0.001787,0.005617,0.005617,0.001787,0.948144,0.530292,0.573100,0.623769,0.886945,0.933050,0.956821,0.824732,0.485751,0.490374,0.933204,0.956907,0.825236
2,0.065574,0.629993,0.304434,hvd.32044106314859,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1896.0,2922091.0,1896.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.117871,0.011956,0.011483,0.012232,0.071876,0.167480,0.218220,0.280590,0.132470,0.490374,0.171549,0.223894,0.287773
3,0.400788,0.038419,0.560793,uc1.$b29323,0.051752,0.055923,0.005905,0.0,0.003112,0.003986,0.000295,0.001169,1825.0,34661558.0,1825.0,0.000049,0.000000,0.000295,0.000049,0.000000,0.255345,0.137686,0.129904,0.107775,0.447032,0.410107,0.447425,0.280590,0.744932,0.490374,0.411484,0.449238,0.287773
4,0.028457,0.594895,0.376647,mdp.39015076816662,0.044464,0.059229,0.005407,0.0,0.000000,0.000000,0.007952,0.007952,1882.0,10080644.0,1882.0,0.000000,0.000000,0.007952,0.000000,0.000000,0.666472,0.112096,0.140452,0.095192,0.071876,0.969351,0.218220,0.280590,0.132470,0.490374,0.969425,0.223894,0.287773


In [7]:
#start by merging year from Hathitrust data
matched_volumes = matched_volumes.merge(hathitrust_data[['HTID', 'Year']], on = 'HTID', how = 'inner')

In [8]:
matched_volumes = matched_volumes[matched_volumes['Year'] <= 1800]

In [9]:
matched_volumes

,Title,Title.1,HTID,ESTC System No.,Match Score,Year
20,Transactions,Instructions,coo.31924003457953,6237091,66.666667,1791.0
33,Vegetable culture for amateurs,Prayers for children,coo.31924003303660,6274137,52.000000,1800.0
84,"Grand Council Royal and Select Masters, Englan...","To Edward Wheler, Esqr. &ca. Council, fort Wil...",coo.31924016286381,6043209,52.252252,1800.0
94,Cucumber culture for amateurs,Prayers for children,coo.31924019147622,6274137,57.142857,1800.0
107,The Law list,The Daily post,coo.31924006955771,6072265,61.538462,1700.0
...,...,...,...,...,...,...
162448,The first book of architecture,The first book of architecture,ucbk.ark+=28722=h29p8v,6152011,100.000000,1721.0
162457,The first book of architecture,The first book of architecture,ucbk.ark+=28722=h20d1n,6152011,100.000000,1716.0
162467,The lives and characters of the English dramat...,An essay on the dramatic character of Sir John...,emu.010002482099,6245451,67.272727,1699.0
162468,"A short account of that part of Africa, inhabi...",A brief account of the present state of the Af...,emu.010002630659,6328251,66.115702,1768.0


In [10]:
matched_volumes.rename(columns={'Title': 'title_hathi', 'ESTC System No.': 'estc_id', 'Match Score': 'match_score'}, inplace=True)
matched_volumes.drop(columns='Title.1', inplace=True)
matched_volumes.head()

,title_hathi,HTID,estc_id,match_score,Year
20,Transactions,coo.31924003457953,6237091,66.666667,1791.0
33,Vegetable culture for amateurs,coo.31924003303660,6274137,52.000000,1800.0
84,"Grand Council Royal and Select Masters, Englan...",coo.31924016286381,6043209,52.252252,1800.0
94,Cucumber culture for amateurs,coo.31924019147622,6274137,57.142857,1800.0
107,The Law list,coo.31924006955771,6072265,61.538462,1700.0


In [11]:
matched_volumes_estc = pd.merge(matched_volumes, estc_data, on = 'estc_id', how = "inner")

In [12]:
matched_volumes_estc

,title_hathi,HTID,estc_id,match_score,Year,ESTC Citation No.,Author - personal,title_estc,Publisher/year,Physical descr.,General note,Citation/references,Surrogates,Loc. of filmed copy,Subject,Added name,Copies - Brit.Isles,Copies - N.America,Electronic location,Variant title,Uncontrolled note,Author - corporate,Uniform title,Corporate subject,Copies - Other,Edition,Person as subject,Added Title,Genre/form,Title as subject,Copies - Europe,Collective title,Conference subject,Index/finding aids,Numbering note,Dates of pub.,Publicat. frequency,Continued by,Other editions,Supplement to,Link note,Related item,Supplement details,Continues,Meeting name,Supplement note,System details,Issued with,Translated as,Loc. of original,"Summary, etc.",Former pub. freq.,Abbreviated title,Translation of,Issuing body,Merged with,Absorbed in part,Bibliography etc.,Series,Absorbed by,Continued in pt. by,Related record,Absorbed,Formed by,Changed back to,Continues in part,Geographic coverage,Separated from,Language note
0,Transactions,coo.31924003457953,6237091,66.666667,1791.0,T37289,NaN,Instructions,"[London, 1706?]","[2],65[1]p. ; 2⁰.",Undated instructions issued by Prince George o...,NaN,NaN,NaN,NaN,NaN,British Library,NaN,NaN,NaN,NaN,England and Wales. Royal Navy.,Instructions. 1706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Transactions,coo.31924062393214,6237091,66.666667,1800.0,T37289,NaN,Instructions,"[London, 1706?]","[2],65[1]p. ; 2⁰.",Undated instructions issued by Prince George o...,NaN,NaN,NaN,NaN,NaN,British Library,NaN,NaN,NaN,NaN,England and Wales. Royal Navy.,Instructions. 1706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Transactions,mdp.39015055639655,6237091,66.666667,1800.0,T37289,NaN,Instructions,"[London, 1706?]","[2],65[1]p. ; 2⁰.",Undated instructions issued by Prince George o...,NaN,NaN,NaN,NaN,NaN,British Library,NaN,NaN,NaN,NaN,England and Wales. Royal Navy.,Instructions. 1706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Vegetable culture for amateurs,coo.31924003303660,6274137,52.000000,1800.0,T179380,"Wesley, John, 1703-1791.",Prayers for children.,"Bristol : printed by William Pine, 1772.","[2],5-22p. ; 12⁰.",Anonymous. By John Wesley.,"Baker, 281",NaN,NaN,NaN,NaN,"The John Rylands Library, The University of Ma...",NaN,NaN,NaN,Half-title called for?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cucumber culture for amateurs,coo.31924019147622,6274137,57.142857,1800.0,T179380,"Wesley, John, 1703-1791.",Prayers for children.,"Bristol : printed by William Pine, 1772.","[2],5-22p. ; 12⁰.",Anonymous. By John Wesley.,"Baker, 281",NaN,NaN,NaN,NaN,"The John Rylands Library, The University of Ma...",NaN,NaN,NaN,Half-title called for?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,The Bishop of Worcester's answer to Mr. Locke'...,uc1.b3356430,6139636,68.452381,1697.0,R40014,"Locke, John, 1632-1704.","A letter to Edward Ld Bishop of Worcester, con...",London : Printed for A[wnsham]. and J[ohn]. Ch...,"[4], 227, [1] p. ; 8⁰.","['Printers’ names fr

In [13]:
#Match thresholds
matched_75 = matched_volumes_estc[matched_volumes_estc['match_score'] >= 75]
matched_80 = matched_volumes_estc[matched_volumes_estc['match_score'] >= 80]
matched_85 = matched_volumes_estc[matched_volumes_estc['match_score'] >= 85]
matched_90 = matched_volumes_estc[matched_volumes_estc['match_score'] >= 90]

print('Number of matched volumes at each threshold: 75 - ' + str(len(matched_75)) + ' 80 - ' + str(len(matched_80)) + ' 85 - ' + str(len(matched_85)) + ' 90 - ' + str(len(matched_90)))

Number of matched volumes at each threshold: 75 - 5550 80 - 3823 85 - 2571 90 - 1832
